In [1]:
from typing import TypedDict, Optional, List, Dict, Any
import numpy as np
import pyrealsense2 as rs
import cv2
import base64
from PIL import Image
import io
from langchain_openai import ChatOpenAI
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_community.document_loaders.image import UnstructuredImageLoader
from langchain_core.tools import tool
from langgraph.graph import MessagesState, StateGraph, START, END
from langgraph.prebuilt import ToolNode, tools_condition
from dataclasses import dataclass

In [17]:
from color_frame_stream import RealSenseStream, ColorFrame

# Define the tools
def capture_and_encode_frame() -> str:
    """Captures and encodes a frame from RealSense Camera"""
    camera = RealSenseStream()
    try:
        color_frame = camera.streaming_color_frame()
        if color_frame.error:
            return f"Failed to capture frame: {color_frame.error}"
        
        pil_image = Image.fromarray(color_frame.image)
        buffer = io.BytesIO()
        pil_image.save(buffer, format="JPEG")
        return base64.b64encode(buffer.getvalue()).decode("utf-8")
    except Exception as e:
        return f"Error capturing frame: {str(e)}"
    finally:
        camera.stop()

ImportError: cannot import name 'RealSenseStream' from 'color_frame_stream' (/home/ari/camera/color_frame_stream.py)

In [3]:
class VisionAnalysisState(TypedDict):
    base64_image: str
    messages: str
    error: Optional[str]

In [18]:
# example_notebook.ipynb
from camera_manager import CameraManager
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

# Create camera manager instance
cam = CameraManager()

# Get a single frame
frame = cam.get_current_frame()
if frame is not None:
    plt.imshow(frame)
    plt.axis('off')
    plt.show()

# Or continuously capture frames
import time
try:
    for _ in range(10):  # Get 10 frames
        frame = cam.get_current_frame()
        if frame is not None:
            plt.imshow(frame)
            plt.axis('off')
            plt.show()
        time.sleep(0.1)
finally:
    cam.stop()


In [21]:
from camera_manager import CameraManager
import time
import matplotlib.pyplot as plt
%matplotlib inline

# Get access to the running camera manager
cam = CameraManager()

# Get and display current frames
for _ in range(10):  # Get 10 frames
    current_frame = cam.get_current_frame()
    if current_frame is not None:
        plt.clf()
        plt.imshow(current_frame)
        plt.axis('off')
        plt.show()
    time.sleep(0.1)

In [ ]:
from camera_manager import CameraManager
import time
import matplotlib.pyplot as plt
%matplotlib inline

# Get access to the running camera manager
cam = CameraManager()

# Get and display current frames
for i in range(10):  # Get 10 frames
    current_frame = cam.get_current_frame()
    print(f"Frame {i}: {'Found' if current_frame is not None else 'None'}")  # Debug print
    
    if current_frame is not None:
        plt.clf()
        plt.imshow(current_frame)
        plt.axis('off')
        plt.show()
    else:
        print("No frame available")
    time.sleep(0.1)